In [74]:
from pymongo import MongoClient
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import folium
import json

## Query tweets

PyMongo: http://api.mongodb.com/python/current/tutorial.html

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [4]:
db = client.twitter_small
tweets = db.tweets

In [7]:
for doc in tweets.find({"place.place_type": "poi"}):
    pprint.pprint(doc)

{'_id': ObjectId('5b2e7a232b5a5caaaa08250f'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Jan 01 04:59:59 +0000 2018',
 'display_text_range': [0, 16],
 'entities': {'hashtags': [{'indices': [11, 16], 'text': '????'}],
              'media': [{'display_url': 'pic.twitter.com/ZddcHwiJsY',
                         'expanded_url': 'https://twitter.com/rallysclub/status/947693866198560770/photo/1',
                         'id': 947693816797990912,
                         'id_str': '947693816797990912',
                         'indices': [17, 40],
                         'media_url': 'http://pbs.twimg.com/media/DSbigVoUEAA4Rh8.jpg',
                         'media_url_https': 'https://pbs.twimg.com/media/DSbigVoUEAA4Rh8.jpg',
                         'sizes': {'large': {'h': 2048,
                                             'resize': 'fit',
                                             'w': 1536},
                                   'medium': {'h': 1200,
             

KeyboardInterrupt: 

In [185]:
within_tweets = tweets.find({"place.better_bounding_box": 
                             { "$geoWithin": 
                              { "$centerSphere": [[ -93.278, 44.963 ], 100 / 3963.2] } }
                            },
                           {"place.centroid.coordinates": 1,
                            "place.place_type": 2,
                            "place.full_name": 3,
                            "text": 3})
within_tweets.count()

967

In [155]:
near_tweets = tweets.find({"coordinates": 
                           { "$near": 
                            { "$geometry": 
                             { "type": "Point",  "coordinates": [ -93.278, 44.963 ] }, 
                             "$minDistance": 0, 
                             "$maxDistance": 15000 }
                           }
                          },
                         {"coordinates.coordinates": 1,
                          "place.place_type": 2,
                          "place.full_name": 3,
                          "text": 3})
#type(near_tweets) # this is a pymongo cursor object
near_tweets.count()

36

In [184]:
tweets_df = pd.DataFrame(list(within_tweets))
print(tweets_df)

                          _id  \
0    5b2e987b2b5a5caaaa0d4ce5   
1    5b2e84972b5a5caaaa0b23ff   
2    5b2e7dda2b5a5caaaa09ea34   
3    5b2e7f3e2b5a5caaaa0a3880   
4    5b2e9a9a2b5a5caaaa0d7b1c   
5    5b2e7a422b5a5caaaa087183   
6    5b2e7acb2b5a5caaaa08e3c1   
7    5b2e7b202b5a5caaaa090e48   
8    5b2e7b672b5a5caaaa092d29   
9    5b2e7b7f2b5a5caaaa093665   
10   5b2e7ccd2b5a5caaaa09a473   
11   5b2e84d72b5a5caaaa0b2d66   
12   5b2e85412b5a5caaaa0b3c9b   
13   5b2e8ca32b5a5caaaa0c2771   
14   5b2e8e7b2b5a5caaaa0c5a9f   
15   5b2e97a02b5a5caaaa0d39cf   
16   5b2e98292b5a5caaaa0d45e2   
17   5b2e9a072b5a5caaaa0d6eb6   
18   5b2e8a2a2b5a5caaaa0bdec8   
19   5b2e9a002b5a5caaaa0d6e1e   
20   5b2e9cfa2b5a5caaaa0dac82   
21   5b2e9d822b5a5caaaa0db764   
22   5b2e95452b5a5caaaa0d043d   
23   5b2e80e42b5a5caaaa0a88c2   
24   5b2e7afb2b5a5caaaa08fd04   
25   5b2e7b572b5a5caaaa092682   
26   5b2e7c692b5a5caaaa0986b7   
27   5b2e89ab2b5a5caaaa0bcfaf   
28   5b2e7a232b5a5caaaa08277d   
29   5b2e7

## Map tweets

Folium: http://folium.readthedocs.io/en/latest/

In [192]:
# Initialize map
m = folium.Map(
    location=[45.4, -93.5],
    zoom_start=8,
    tiles='Stamen Terrain'
)

# Iterate over tweets, adding markers to map for each tweet
for index, row in tweets_df.iterrows():
    #coords_dict = dict(row['coordinates'])
    #long = coords_dict['coordinates'][0]
    #lat = coords_dict['coordinates'][1]
    
    place_dict = dict(row['place'])
    place_name = place_dict['full_name']
    long = place_dict['centroid']['coordinates'][0]
    lat = place_dict['centroid']['coordinates'][1]
    
    popup_text = "<b>" + str(place_name) + ": </b>" + str(row['_id'])
    
    folium.Marker(
        location=[lat, long],
        #popup = popup_text,
        icon=folium.Icon(icon='cloud')
    ).add_to(m)

# Display map
m